
# Surface Measurements — XLOT/DFIT Plot Notebook

This notebook focuses **only on surface time/pressure measurements**, applying the same diagnostics
you already use (Bourdet, √t, and G-function), plus **intersection methods** (with fit-line overlays)
adapted to the *surface* reference frame.

> **How to use**  
> 1. Run the **Setup** cell to import your local modules.  
> 2. Prepare your input arrays in the **Inputs** cell (or import from your pipeline):
>    - `time_S`: `pd.DatetimeIndex` or Series of surface timestamps
>    - `p_surface_corr`: surface pressure series (already corrected if you prefer)
>    - `cycles`: list of per-cycle dicts with at least `t_shut_in_surface`, `t_end_surface`, and (if available) `t_pump_start_surface`
>    - `CAP_S`: numeric max seconds for analysis window (e.g., 180)
>    - `MIN_T_S_FOR_PICK`: minimum seconds for closure pick (e.g., 0 or 90)
> 3. Run the **Per-cycle analysis (SURFACE frame)** cell to generate plots.
>
> Notes:
> - The **surface** frame means: **no lag shift** is applied to the anchor times.  
> - Pump time `tp_s` is computed from the surface timestamps (if available).  
> - The overlays are defensive: if your intersection functions don't return fit coefficients, the lines are skipped safely.


In [ ]:

# ===============
# Setup & imports
# ===============
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Local modules expected in your working directory / PYTHONPATH
import time_difference
import well_corrections
import closure_analysis
import plotting

# For nicer plots (optional)
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['axes.grid'] = True


In [ ]:

# ======
# Inputs
# ======
# Provide/override these from your pipeline as needed.

# Surface time series (pd.DatetimeIndex or Series of timestamps)
# Example placeholders:
# time_S = pd.to_datetime(df_surface['timestamp'])
# p_surface_corr = df_surface['P_surface_bar_corr'].to_numpy(float)

try:
    time_S
    p_surface_corr
except NameError:
    # Minimal placeholders so the notebook runs without your data (feel free to overwrite)
    rng = pd.date_range('2025-01-01 00:00:00', periods=400, freq='S')
    time_S = rng
    # Fake pressure drop after a 'shut-in' around index 100 for demo purposes
    p_surface_corr = np.linspace(400, 300, len(rng)) + np.random.normal(0, 0.3, len(rng))

# Cycles list with at least the keys shown below
# Each cycle:
#   - 't_shut_in_surface': Timestamp
#   - 't_end_surface'    : Timestamp
#   - 't_pump_start_surface' (optional): Timestamp (for tp_s)
try:
    cycles
except NameError:
    # Demo cycle on the placeholder series
    cycles = [{
        't_shut_in_surface': pd.Timestamp('2025-01-01 00:01:40'),  # index ~100
        't_end_surface':     pd.Timestamp('2025-01-01 00:03:40'),
        'ended_by':          'demo',
        't_pump_start_surface': pd.Timestamp('2025-01-01 00:01:10'),
    }]

# Analysis caps
try:
    CAP_S
except NameError:
    CAP_S = 180.0

try:
    MIN_T_S_FOR_PICK
except NameError:
    MIN_T_S_FOR_PICK = 0.0



## Helper — build shut-in series in **surface** frame


In [ ]:

def build_shut_in_series_surface(time_surface, p_surface, t_shut_in_surface):
    """
    Wrapper around your existing build logic, but **stays in SURFACE clock**.
    Returns (ts_ref, p_ref) where ts_ref is seconds since shut-in (surface frame).
    """
    # Reuse your function if it accepts any time array + anchor;
    # otherwise, replicate a minimal version here:
    return closure_analysis.build_shut_in_series(time_surface, p_surface, t_shut_in_surface)



## Per-cycle analysis (SURFACE frame)


In [ ]:

print("\nPer-cycle closure analysis on **surface** measurements (Bourdet, √t, G within 0–%d s):" % CAP_S)

for i, cyc in enumerate(cycles, 1):
    # Anchors in **surface** frame (no lag applied)
    t_shut_ref = cyc['t_shut_in_surface']
    t_end_ref  = cyc['t_end_surface']

    # Build + cap falloff series (surface)
    ts_ref, p_ref = build_shut_in_series_surface(time_S, p_surface_corr, t_shut_ref)
    t_end_rel     = (pd.to_datetime(t_end_ref) - pd.to_datetime(t_shut_ref)).total_seconds()
    t_hard_cap    = min(float(t_end_rel), float(CAP_S))
    keep          = (ts_ref <= t_hard_cap)
    ts_ref, p_ref = ts_ref[keep], p_ref[keep]

    # Require some usable falloff
    if len(ts_ref) < 3 or (ts_ref[-1] if len(ts_ref) else 0) < 120:
        print(f"  Cycle {i}: skipped (too short falloff: {ts_ref[-1] if len(ts_ref) else 0:.0f}s).")
        continue

    # --- Bourdet derivative pick (global minimum) ---
    t_log, dP_dlogt = closure_analysis.bourdet_derivative(
        ts_ref, p_ref, smooth_win=None, max_t_s=CAP_S
    )
    i_brd = closure_analysis.suggest_closure_from_bourdet(
        t_log, dP_dlogt, min_t_s=MIN_T_S_FOR_PICK, max_t_s=CAP_S
    )

    # --- √t diagnostic (QC) ---
    x_sqrt, p_srt, dpdx = closure_analysis.derivative_vs_sqrt_time(
        ts_ref, p_ref, max_t_s=CAP_S
    )
    i_srt = closure_analysis.suggest_closure_from_srt(
        x_sqrt, p_srt, dpdx, min_t_s=MIN_T_S_FOR_PICK, max_t_s=CAP_S
    )

    # --- G-function diagnostic (Barree/Nolte) ---
    # Approx pumping time per cycle: tp = shut-in - pump-start (surface clock)
    try:
        tp_s = (cyc['t_shut_in_surface'] - cyc['t_pump_start_surface']).total_seconds()
    except Exception:
        tp_s = 0.0
    if not np.isfinite(tp_s) or tp_s <= 0:
        tp_s = max(1.0, ts_ref[0] if len(ts_ref) else 1.0)  # safe fallback

    G = closure_analysis.g_function_high_efficiency(ts_ref, tp_s)
    semilog_dP = closure_analysis.semilog_derivative(G, p_ref)
    i_g = None
    if len(G) >= 3 and np.isfinite(semilog_dP).any():
        mwin = (
            (ts_ref >= float(MIN_T_S_FOR_PICK)) &
            (ts_ref <= float(CAP_S)) &
            np.isfinite(semilog_dP)
        )
        if mwin.any():
            idx = np.where(mwin)[0]
            i_g = idx[np.nanargmin(semilog_dP[idx])]

    # --- Plot √t (pressure on left; dP/d√t & √t·dP/d√t on right), with marker ---
    figSRT, (ax_left_srt, ax_right_srt) = plotting.plot_srt(
        x_sqrt, p_srt, dpdx, i_cl=i_srt, cap_s=CAP_S
    )

    # ===========================
    # √t INTERSECTION METHOD + overlay
    # ===========================
    try:
        res_srt = closure_analysis.fcp_by_sqrt_intersection(
            ts_ref, p_ref, max_t_s=CAP_S, min_left=8, min_right=8
        )
    except Exception:
        res_srt = {"ok": False}

    if res_srt.get("ok"):
        # Print numeric result
        try:
            cp_bar = float(res_srt.get("closure_pressure_bar"))
            ct_s   = float(res_srt.get("closure_time_s"))
        except Exception:
            cp_bar, ct_s = (np.nan, np.nan)

        if np.isfinite(cp_bar) and np.isfinite(ct_s):
            print(f"            [Cycle {i}] FCP (√t inters.) ≈ {cp_bar:.2f} bar at t ≈ {ct_s:.0f} s")

        # Overlay fitted lines on P vs √t panel
        left_fit  = res_srt.get("left_fit",  (None, None))
        right_fit = res_srt.get("right_fit", (None, None))

        # Normalize coefficients to floats safely
        try:
            mL, bL = (float(left_fit[0]), float(left_fit[1]))
        except Exception:
            mL, bL = (np.nan, np.nan)
        try:
            mR, bR = (float(right_fit[0]), float(right_fit[1]))
        except Exception:
            mR, bR = (np.nan, np.nan)

        # Determine x-range split at closure (in √t domain)
        if np.isfinite(ct_s) and ct_s >= 0:
            x_cl = np.sqrt(ct_s)
            xL = x_sqrt[x_sqrt <= x_cl]
            xR = x_sqrt[x_sqrt >= x_cl]
        else:
            x_cl = np.nan
            xL, xR = x_sqrt, x_sqrt

        # Draw lines only if finite
        if np.isfinite(mL) and np.isfinite(bL) and len(xL) > 1:
            ax_left_srt.plot(xL, mL * xL + bL, ls="--", lw=1.2, c="k", label="√t left fit")
        if np.isfinite(mR) and np.isfinite(bR) and len(xR) > 1:
            ax_left_srt.plot(xR, mR * xR + bR, ls="--", lw=1.2, c="k", label="√t right fit")

        # Mark intersection point
        if np.isfinite(x_cl) and np.isfinite(cp_bar):
            ax_left_srt.scatter([x_cl], [cp_bar], s=20, c="k", zorder=5)

        try:
            ax_left_srt.legend(loc="best", fontsize=8)
        except Exception:
            pass

    # --- Plot Bourdet (pressure on left; dP/dln t on right), with marker ---
    figBRD, axBRD = plotting.plot_bourdet(
        t_log, dP_dlogt,
        p=p_ref,           # pressure samples
        p_times=ts_ref,    # their time base (sec since shut-in)
        i_cl=i_brd,
        cap_s=CAP_S
    )
    if i_brd is not None:
        axBRD.set_title(f"Cycle {i} — (surface) Bourdet min at ~{t_log[i_brd]:.0f}s, P≈{p_ref[i_brd]:.2f} bar")
    else:
        axBRD.set_title(f"Cycle {i} — (surface) no Bourdet min in [{int(MIN_T_S_FOR_PICK)}, {int(CAP_S)}] s")

    # --- Plot G-function (pressure on left; semilog & normal derivs on right), with marker ---
    figG, axG = plotting.plot_gfunction(
        ts_seconds=ts_ref,
        p=p_ref,
        tp_seconds=tp_s,
        p_times=ts_ref,
        i_cl=i_g,
        cap_s=CAP_S
    )
    if i_g is not None:
        axG.set_title(
            f"Cycle {i} — (surface) G-function (tp={tp_s:.0f}s), min semilog dP at t≈{ts_ref[i_g]:.0f}s, "
            f"P≈{p_ref[i_g]:.2f} bar"
        )
    else:
        axG.set_title(f"Cycle {i} — (surface) G-function (tp={tp_s:.0f}s), no min in [{int(MIN_T_S_FOR_PICK)}, {int(CAP_S)}] s")

    # =======================
    # G INTERSECTION METHOD + overlay
    # =======================
    try:
        res_g = closure_analysis.fcp_by_g_intersection(
            ts_ref, p_ref, tp_seconds=tp_s, max_t_s=CAP_S, min_left=8, min_right=8
        )
    except Exception:
        res_g = {"ok": False}

    if res_g.get("ok"):
        # Print numeric result
        try:
            cp_bar_g = float(res_g.get("closure_pressure_bar"))
            ct_s_g   = float(res_g.get("closure_time_s"))
        except Exception:
            cp_bar_g, ct_s_g = (np.nan, np.nan)

        if np.isfinite(cp_bar_g) and np.isfinite(ct_s_g):
            print(f"            [Cycle {i}] FCP (G inters.)  ≈ {cp_bar_g:.2f} bar at t ≈ {ct_s_g:.0f} s (tp={tp_s:.0f}s)")

        # Coefficients
        left_fit_g  = res_g.get("left_fit",  (None, None))
        right_fit_g = res_g.get("right_fit", (None, None))
        try:
            mL_g, bL_g = (float(left_fit_g[0]), float(left_fit_g[1]))
        except Exception:
            mL_g, bL_g = (np.nan, np.nan)
        try:
            mR_g, bR_g = (float(right_fit_g[0]), float(right_fit_g[1]))
        except Exception:
            mR_g, bR_g = (np.nan, np.nan)

        # Closure G-value
        G_cl = res_g.get("closure_G", None)
        if G_cl is not None:
            try:
                G_cl = float(G_cl)
            except Exception:
                G_cl = np.nan
        else:
            # Derive from closure time by nearest index
            if np.isfinite(ct_s_g):
                k = int(np.argmin(np.abs(ts_ref - ct_s_g)))
                G_cl = G[k] if 0 <= k < len(G) else np.nan
            else:
                G_cl = np.nan

        # Split G range
        if np.isfinite(G_cl):
            GL = G[G <= G_cl]
            GR = G[G >= G_cl]
        else:
            GL, GR = G, G

        # Draw lines only if finite
        if np.isfinite(mL_g) and np.isfinite(bL_g) and len(GL) > 1:
            axG.plot(GL, mL_g * GL + bL_g, ls="--", lw=1.2, c="k", label="G left fit")
        if np.isfinite(mR_g) and np.isfinite(bR_g) and len(GR) > 1:
            axG.plot(GR, mR_g * GR + bR_g, ls="--", lw=1.2, c="k", label="G right fit")

        # Mark intersection
        if np.isfinite(G_cl) and np.isfinite(cp_bar_g):
            axG.scatter([G_cl], [cp_bar_g], s=20, c="k", zorder=5)

        try:
            axG.legend(loc="best", fontsize=8)
        except Exception:
            pass

    # -----------------
    # Print the results
    # -----------------
    if i_brd is not None:
        print(f"  Cycle {i}: FCP (Bourdet) ≈ {p_ref[i_brd]:.2f} bar at t ≈ {t_log[i_brd]:.0f} s "
              f"(ended by {cyc.get('ended_by','?')}, window {t_hard_cap:.0f}s)")
    else:
        print(f"  Cycle {i}: no Bourdet minimum found in [{MIN_T_S_FOR_PICK}, {CAP_S}] s.")

    if i_srt is not None:
        print(f"            FCP (√t)      ≈ {p_srt[i_srt]:.2f} bar at t ≈ {x_sqrt[i_srt]**2:.0f} s")

    if i_g is not None:
        print(f"            FCP (G-func)  ≈ {p_ref[i_g]:.2f} bar at t ≈ {ts_ref[i_g]:.0f} s "
              f"(tp={tp_s:.0f}s)")
